# Data Preparation and Transformation of Cyclistic

### Background & Problem

*__Cyclistic__*: A bike-share program that features more than 5,800 bicycles and a network of 692 docking stations across Chicago. That provides access to bikes that can be unlocked from one station and returned to any other station in the system. Likewise, setting itself apart by offering reclining bikes, hand tricycles, and cargo bikes, making bike-share more inclusive to people with disabilities and riders who can’t use a standard two-wheeled bike.

Cyclistic users generally purchase single-ride passes, full-day passes, and annual memberships to ride for leisure, explore the city or commute to work each day. Thus we want explore how annual members and casual riders use Cyclistic bikes diferently.

### KPI's

1. What day of the week are the bikes are rented?
2. What time of day are the bikes being rented the most?
3. Does season’s have an effect on bikes renting?
4. Which area has the most renting recorded? (Suburban, Commercial, Private Area, etc...)

### Data Description

For this analysis, we will use data [__Cyclistic's historicical trip data__](https://divvy-tripdata.s3.amazonaws.com/index.html) from 2019. The data also include the following fields:

| Field             | Description                                  |
|----------------|----------------------------------------------------------------------------------------------|
| trip_id           | Unique identifier of renting transcation     |
| start_time        | Date and time when bikes are rented          |
| end_time          | Date and time when bikes are returned        |
| bikeid            | Unique identifier of each bike               |
| tripduration      | Total rent duration in seconds               |
| from_station_id   | Station ID where bikes where rented          |
| from_station_name | Station name where bikes where rented        |
| to_station_id     | Station ID where bikes where returned        |     
| to_station_name   | Station name where bikes where returned      |
| usertype          | Type of the riders or bike users             |
| gender            | Gender of annual members or subscribers      |
| birthyear         | Age of the annual members or subscribers     |

### Importing Primary Modules
The program heavily relies on [**pandas**](http://pandas.pydata.org/), [**geopy**](https://geopy.readthedocs.io/en/stable/) and [**datetime**](https://docs.python.org/3/library/datetime.html) for data wrangling and transformation.

In [1]:
import pandas as pd
import csv
import numpy as np
from datetime import datetime
import calendar
from time import sleep
from geopy.geocoders import Nominatim

### Data Wrangling

Extracting csv files from 4 quartiles into separate dataframes.

In [2]:
q1 = pd.read_csv("data/Divvy_Trips_2019_Q1.csv")
q2 = pd.read_csv("data/Divvy_Trips_2019_Q2.csv")
q3 = pd.read_csv("data/Divvy_Trips_2019_Q3.csv")
q4 = pd.read_csv("data/Divvy_Trips_2019_Q4.csv")

In [3]:
#q1.head()

In [4]:
#q2.head()

In [5]:
#q3.head()

In [6]:
#q4.head()

#### Renaming q2 column names

Considering that the columns for 2nd quartile have different column names, the following lines of code renames them to match other dataframes.

In [7]:
q1_col = []
for col in q1.columns:
    q1_col.append(col)
q2_col = []
for col in q2.columns:
    q2_col.append(col)

In [8]:
q2.rename(columns=dict(zip(q2_col, q1_col)), inplace=True)
#q2.head()

#### Merging all dataframes into a single one

In [9]:
df = pd.concat([q1, q2, q3, q4], axis=0,sort=False)
df

,trip_id,start_time,end_time,bikeid,tripduration,from_station_id,from_station_name,to_station_id,to_station_name,usertype,gender,birthyear
0,21742443,2019-01-01 00:04:37,2019-01-01 00:11:07,2167,390.0,199,Wabash Ave & Grand Ave,84,Milwaukee Ave & Grand Ave,Subscriber,Male,1989.0
1,21742444,2019-01-01 00:08:13,2019-01-01 00:15:34,4386,441.0,44,State St & Randolph St,624,Dearborn St & Van Buren St (*),Subscriber,Female,1990.0
2,21742445,2019-01-01 00:13:23,2019-01-01 00:27:12,1524,829.0,15,Racine Ave & 18th St,644,Western Ave & Fillmore St (*),Subscriber,Female,1994.0
3,21742446,2019-01-01 00:13:45,2019-01-01 00:43:28,252,"1,783.0",123,California Ave & Milwaukee Ave,176,Clark St & Elm St,Subscriber,Male,1993.0
4,21742447,2019-01-01 00:14:52,2019-01-01 00:20:56,1170,364.0,173,Mies van der Rohe Way & Chicago Ave,35,Streeter Dr & Grand Ave,Subscriber,Male,1994.0
...,...,...,...,...,...,...,...,...,...,...,...,...
704049,25962900,2019-12-31 23:56:13,2020-01-01 00:15:45,2196,"1,172.0",112,Green St & Randolph St,225,Halsted St & Dickens Ave,Subscriber,Male,1981.0
704050,25962901,2019-12-31 23:56:34,2020-01-01 00:22:08,4877,"1,533.0",90,Millennium Park,90,Millennium Park,Subscriber,Male,1992.0
704051,25962902,2019-12-31 23:57:05,2020-01-01 00:05:46,863,520.0,623,Michigan Ave & 8th St,52,Michigan Ave & Lake St,Subscriber,Male,1967.0
704052,25962903,2019-12-31 23:57:11,2020-01-01 00:05:45,2637,514.0,623,Michigan Ave & 8th St,52,Michigan Ave & Lake St,Subscriber,Female,1970.0


#### Resetting Index

In [10]:
df.index = np.arange(1,len(df)+1)

#### Changing column datatype.

In [11]:
df=df.convert_dtypes()
#df.dtypes

#### Creating new columns

The following lines of separates start and end time into subsiquent dimensions: month, weekday, and hourtime.

In [12]:
def time_data(df_col, list_name, data_variable):
    for row in df_col:
        date = datetime.strptime(row, "%Y-%m-%d %H:%M:%S")
        if data_variable == "h":
            list_name.append(date.hour)
        elif data_variable == "wd":
            list_name.append(calendar.day_name[date.weekday()])
        elif data_variable == "m":
            list_name.append(date.strftime("%B"))

In [13]:
hour_list = []
time_data(df['start_time'], hour_list, "h")
weekday_list = []
time_data(df['start_time'], weekday_list, "wd")
month_list = [] 
time_data(df['start_time'], month_list, "m")

In [14]:
df['Time(hour)'] = hour_list
df['Weekday'] = weekday_list
df['Month'] = month_list

In [15]:
df.to_csv("bike_share.csv")

#### Getting geocoordinates of each station.

Using the geopy library, the following lines of code extract the latitude and longitude of each rental station. The dataset contains over 3 million records, the approximate extraction of geodata using an API for each station may take hours and days.  Thus, considering that station name may certainly overlap, the data processing will be divided into the following steps:
* __Step 1__: Removing strings and characters from both __station_name__ columns that may impact the search.
* __Step 2__: Extract unique/distinc station names into new list. 
* __Step 3__: Using the __geopy API__ extract latitude and longitude into separate lists.
* __Step 4__: Combine station, latitude, and longitude into dataframe and/or dictionary.
* __Step 5__: Using for loop get coordinates for each record where station_names match. (Optional)

#### Step 1: Removing strings and characters from both columns

In [16]:
df['from_station_name'] = df['from_station_name'].str.strip('(*)')
df['from_station_name'] = df['from_station_name'].str.strip("(Temp)")
df['to_station_name'] = df['to_station_name'].str.strip('(*)')
df['to_station_name'] = df['to_station_name'].str.strip("(Temp)")

#### Step 2: Extract unique/distinc station names into new list. 

In [17]:
def unique_station_name(list1, unique_list):
    # traverse for all elements
    for x in list1:
        # check if exists in unique_list or not
        if x not in unique_list:
            unique_list.append(x)

In [18]:
unique_station = []
unique_station_name(df['from_station_name'],unique_station)
unique_station_name(df['to_station_name'],unique_station)
#unique_station

In [19]:
len(unique_station)

641

#### Step 3: Extracting latitude and longitude.

In [20]:
def geo_coordinates(station, lat, long, user_agent):
    geolocator = Nominatim(user_agent=user_agent)
    for row in station:
        try:
            location = geolocator.geocode(row)
            lat.append(location.latitude)
            long.append(location.longitude)
        except:
            lat.append(None)
            long.append(None)
            sleep(0.01)

In [21]:
latitude = []
longitude = []
geo_coordinates(unique_station, latitude, longitude, "bike_share")

#### Step 4: Combine station, latitude, and longitude into dataframe and/or dictionary.

In [22]:
#Dictionary
station_location =[{'name': name, 'lat': lat, 'long': long} for name,lat,long in zip(unique_station,latitude,latitude)]

In [23]:
#Dataframe
station_df = pd.DataFrame()
station_df['station_name'] = unique_station
station_df['lat'] = latitude
station_df['long'] = longitude
station_df.to_csv('station_geodata.csv')

#### Step 5: Using loop get coordinates for each record where station_names match. (Optional)

In [24]:
def station_coordinates(df, station_dict, list_lat, list_long):
    for row in df:
        for s in range(0,len(station_dict)):
            if row == station_dict[s]['name']:
                list_lat.append(station_dict[s]['lat'])
                list_long.append(station_dict[s]['long'])

In [25]:
start_lat = []
start_long = []
station_coordinates(df['from_station_name'], station_location, start_lat, start_long)

In [27]:
end_lat = []
end_long = []
station_coordinates(df['to_station_name'], station_location, end_lat, end_long)
#station_coordinates(df, station_location, end_lat, end_long, 'to_station_name')

In [28]:
df['start_lat'] = start_lat
df['start_long'] = start_long
df['end_lat'] = end_lat
df['end_long'] = end_long

In [29]:
df.to_csv("bike_share_with_geodata.csv")

### Data Visualization

The data visualization dashboard and PowerPoint has been created using __Tableau__ and can be found using the following links:
* __Github__: [click here](https://github.com/sergeykh98/Google-DA-Certification-Bike-Share)
* __Tableau Public__: [click here](https://public.tableau.com/app/profile/sergey.khegay/viz/Cyclisticbike-sharedashboard/Cyclisticbike-sharedashboard)